In [1]:
!pip install requirments.txt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 25.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 57.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 1.8 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 17.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.2 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 71.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 18.3 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu1

In [2]:
#all installs
!pip install -q -U trl transformers accelerate git+https://github.com/huggingface/peft.git
!pip install -q datasets bitsandbytes einops wandb
!pip install huggingface_hub

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 375.8/375.8 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.5/10.5 MB 89.2 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.3/365.3 kB 11.8 MB/s eta 0:00:00


In [3]:
import os
import sys
import json
import time
import torch
import pandas as pd

from datasets import load_dataset, Dataset
from huggingface_hub import notebook_login

from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    GenerationConfig,
    TrainingArguments,
    DataCollatorForLanguageModeling,
    Trainer,
    AutoConfig
)

from peft import (
    LoraConfig,
    get_peft_model,
    prepare_model_for_kbit_training,
    PeftConfig,
    PeftModel
)

from accelerate import init_empty_weights, load_checkpoint_and_dispatch

from sentence_transformers import SentenceTransformer

import faiss
import termios
import tty


2025-06-23 08:43:51.466759: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750668231.628986      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750668231.674674      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


**Environment Variable Pytorch uses to tune its CUDA memory allocator(useful for LLMs)**

In [8]:

os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

In [10]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [6]:
#Acceletrator
from accelerate import FullyShardedDataParallelPlugin, Accelerator
from torch.distributed.fsdp.fully_sharded_data_parallel import FullOptimStateDictConfig, FullStateDictConfig
fsdp_plugin = FullyShardedDataParallelPlugin(
    state_dict_config=FullStateDictConfig(offload_to_cpu=True, rank0_only=False),
    optim_state_dict_config=FullOptimStateDictConfig(offload_to_cpu=True, rank0_only=False),
)
accelerator = Accelerator(fsdp_plugin=fsdp_plugin)

**Hugging Face login to use Mistral7b(it's a gated repository)**

In [4]:
!huggingface-cli login



    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: fineG

In [ ]:
# Load all JSON files into dataframes
df1 = pd.read_json("/content/constitution_qa.json")
df2 = pd.read_json("/content/crpc_qa.json")
df3 = pd.read_json("/content/ipc_qa.json")
df4 = pd.read_json("/content/IndicLegalQA Dataset_10K_Revised.json")

# Combine all into a single dataframe
combined_df = pd.concat([df1, df2, df3,df4], ignore_index=True)

# Save to a new JSON file
combined_df.to_json("combined.json", orient="records", indent=4)

In [ ]:
# Load the dataset
with open("combined.json", "r", encoding="utf-8") as f:
    data = json.load(f)  # Ensure your JSON is a list of dicts, e.g., [{"question": "...", "answer": "..."}]

# Format the data
formatted_data = []
for sample in data:
    formatted_text = f"<s>[INST] {sample['question']} [/INST] {sample['answer']}</s>"
    formatted_data.append({"text": formatted_text})

# Convert to Hugging Face Dataset format
dataset = Dataset.from_list(formatted_data)

# Check formatted sample
print(dataset[0])


{'text': '<s>[INST] What is India according to the Union and its Territory? [/INST] India, that is Bharat, shall be a Union of States.</s>'}


In [ ]:
dataset_path = "combined.json" 

data = load_dataset("json", data_files=dataset_path)

Generating train split: 0 examples [00:00, ? examples/s]

**Model Specification and Bits and bytes config(very standard quantization )**

In [ ]:
model_name = "mistralai/Mistral-7B-Instruct-v0.1"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.float16,
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True
)
model.config.use_cache = False

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

**Tokenizer Initiation**

In [9]:
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

tokenizer_config.json:   0%|          | 0.00/2.10k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

**PEFT(lora config again standard can be found in the documentation)**

In [ ]:

peft_config = LoraConfig(
    lora_alpha=32,
    lora_dropout=0.05,
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "down_proj", "up_proj"]
)
peft_model = get_peft_model(model, peft_config)


**Training Hyperparameters
,Can use the commented down values to get better results(requires more training time)**

In [ ]:

output_dir = "Mistral7binstruct_lawyer"
per_device_train_batch_size = 2 #4
gradient_accumulation_steps = 4
optim = "paged_adamw_32bit"
save_steps = 10
logging_steps = 10
learning_rate = 2e-4
max_grad_norm = 0.3
max_steps = 250 #100 #500
warmup_ratio = 0.03
lr_scheduler_type = "cosine" 

'training_arguments = TrainingArguments(\n    output_dir=output_dir,\n    per_device_train_batch_size=per_device_train_batch_size,\n    gradient_accumulation_steps=gradient_accumulation_steps,\n    optim=optim,\n    save_steps=save_steps,\n    logging_steps=logging_steps,\n    learning_rate=learning_rate,\n    fp16=True,\n    max_grad_norm=max_grad_norm,\n    max_steps=max_steps,\n    warmup_ratio=warmup_ratio,\n    group_by_length=True,\n    lr_scheduler_type=lr_scheduler_type,\n    push_to_hub=False\n)'

In [ ]:
def tokenize_function(example):
    return tokenizer(
        example["text"],
        truncation=True,
        padding="max_length",  # Or use padding=True for dynamic padding
        max_length=256,        # You can increase this for longer inputs
    )

# Apply the tokenizer to your dataset
tokenized_dataset = dataset.map(tokenize_function, batched=True)

#remvoe the text column from the dataset(memory efficient)

tokenized_dataset = tokenized_dataset.remove_columns(["text"])


Map:   0%|          | 0/24543 [00:00<?, ? examples/s]

In [14]:
training_arguments.remove_unused_columns = False


NameError: name 'training_arguments' is not defined

In [16]:
training_arguments = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    fp16=True,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=True,
    lr_scheduler_type=lr_scheduler_type,
    push_to_hub=False,
    remove_unused_columns=False,  # ADD THIS!
)


**Datacollator assumess the data is already tokenized and pads the sequence dynamically and prepare tensors like input_dims,attention_mask etc**
,mlm = masked language Model should be False as we are dealing with Mistral/falcon/gpt2 type of model which predicts the next words through the previous words not like BERT which masks random tokens in the training.

In [ ]:
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False  # MLM should be False for causal language modeling like Mistral/Falcon etc
)



Define your Trainer

In [ ]:
trainer = Trainer(
    model=peft_model,
    args=training_arguments,
    train_dataset=tokenized_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
)


/tmp/ipython-input-24-1800042336.py:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [25]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: idealsteve001 (idealsteve001-manipal-academy-of-higher-education-dubai) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss
10,3.002500
20,2.092700
30,1.616900
40,1.516800
50,1.505600
60,1.508100
70,1.474600
80,1.461000
90,1.369300
100,1.388000


TrainOutput(global_step=250, training_loss=1.504821559906006, metrics={'train_runtime': 3004.3876, 'train_samples_per_second': 0.666, 'train_steps_per_second': 0.083, 'total_flos': 2.2359343890432e+16, 'train_loss': 1.504821559906006, 'epoch': 0.08148631029986962})

**Save this model this is only the LORA adapters from the Mistral model you will need to call the Mistral Model again and then combine these two to get to our fine-tunned model.**

In [34]:
model_save_path = "Mistral7B_lawyer_model"
trainer.save_model(model_save_path)
tokenizer.save_pretrained(model_save_path)


('Mistral7B_lawyer_model/tokenizer_config.json',
 'Mistral7B_lawyer_model/special_tokens_map.json',
 'Mistral7B_lawyer_model/chat_template.jinja',
 'Mistral7B_lawyer_model/tokenizer.model',
 'Mistral7B_lawyer_model/added_tokens.json',
 'Mistral7B_lawyer_model/tokenizer.json')

**!!This saves teh entire Mistral+LORA adapters model(big file = 3.9GB) would not recommend to execute this code as it takes a while and eats up all the sapce and needs a lot of VRAM to load.**

In [33]:
# If this is still your current trained model:
merged_model = peft_model.merge_and_unload()

# Save the merged model
merged_model.save_pretrained("Mistral7B_lawyer_merged")
tokenizer.save_pretrained("Mistral7B_lawyer_merged")



/usr/local/lib/python3.11/dist-packages/peft/tuners/lora/bnb.py:348: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(


('Mistral7B_lawyer_merged/tokenizer_config.json',
 'Mistral7B_lawyer_merged/special_tokens_map.json',
 'Mistral7B_lawyer_merged/chat_template.jinja',
 'Mistral7B_lawyer_merged/tokenizer.model',
 'Mistral7B_lawyer_merged/added_tokens.json',
 'Mistral7B_lawyer_merged/tokenizer.json')

**Optional: Remove all the cache accumulated**

In [30]:
!rm -rf wandb/
!rm -rf Mistral7B_lawyer_model/checkpoint-*
!rm -rf ~/.cache/huggingface/transformers
!rm -rf ~/.cache/huggingface/datasets



**RAG Document Setup**

In [ ]:

dataset = load_dataset("viber1/indian-law-dataset", split="train")
print(dataset[0])  

README.md:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

train.jsonl:   0%|          | 0.00/14.4M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/24607 [00:00<?, ? examples/s]

{'Instruction': 'What is the difference between a petition and a plaint in Indian law?', 'Response': "A petition is a formal request submitted to a courttribunalor authority to seek a specific remedy or relief. It is commonly used for various purposessuch as filing a writ petition in the High Court or submitting a petition for divorce. On the other handa plaint is a formal written statement of a plaintiff's claim in a civil lawsuit. The key difference is that a petition is more versatile and can be used for various legal matterswhile a plaint is specific to civil cases."}


In [42]:
rag_docs = [
    f"Q: {item['Instruction']}\nA: {item['Response']}"
    for item in dataset
]


In [ ]:
embed_model = SentenceTransformer("all-MiniLM-L6-v2")
embeddings = embed_model.encode(rag_docs, convert_to_numpy=True, show_progress_bar=True)


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/769 [00:00<?, ?it/s]

**Inference**

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import PeftModel
import torch

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained("Mistral7B_lawyer_model", trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token  # Optional for safety

# Quantization config for Mistral (must match training)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.float16,
)

# Load base Mistral model in 4-bit
base_model = AutoModelForCausalLM.from_pretrained(
    "mistralai/Mistral-7B-Instruct-v0.1",
    device_map="auto",
    quantization_config=bnb_config,
    trust_remote_code=True
)

# Load LoRA adapter on top
model = PeftModel.from_pretrained(base_model, "Mistral7B_lawyer_model")
model.eval()


**RAG Pipeline**

In [ ]:

# === CONFIG ===
model_path = "/content/Mistral7B_lawyer_merged"  # or mistral + LoRA if needed
tokenizer = AutoTokenizer.from_pretrained(model_path)

model = AutoModelForCausalLM.from_pretrained(
    model_path,
    device_map="auto",
    torch_dtype=torch.float16,
    low_cpu_mem_usage=True,
    offload_folder="offload"
)
model.eval()

# === RAG EMBEDDING SETUP ===
from sentence_transformers import SentenceTransformer

# rag_docs = [f"Q: {item['Instruction']}\nA: {item['Response']}" for item in dataset]
# embeddings = embed_model.encode(rag_docs, convert_to_numpy=True)

index = faiss.IndexFlatL2(embeddings.shape[1])
index.add(embeddings)

def retrieve_context(query, top_k=3):
    query_vec = embed_model.encode([query])
    D, I = index.search(query_vec, top_k)
    return [rag_docs[i] for i in I[0]]

def rag_chat(query):
    context = retrieve_context(query)
    prompt = (
        "<s>[INST] You are a legal assistant. Use the following context to answer the query.\n\n"
        + "\n\n".join(context) +
        f"\n\nQuestion: {query}\nAnswer: [/INST]"
    )
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=256,
            do_sample=True,
            temperature=0.7,
            top_p=0.9,
            repetition_penalty=1.1,
            pad_token_id=tokenizer.eos_token_id,
        )
    return tokenizer.decode(outputs[0], skip_special_tokens=True).split("[/INST]")[-1].strip()



**ChatBot**

In [60]:
def wait_for_keypress():
    #Detects single keypress like Esc
    fd = sys.stdin.fileno()
    old_settings = termios.tcgetattr(fd)
    try:
        tty.setraw(fd)
        key = sys.stdin.read(1)
    finally:
        termios.tcsetattr(fd, termios.TCSADRAIN, old_settings)
    return key

def run_chat():
    print("🧑‍⚖️ LawBot RAG Chat (type 'exit' to quit)\n")
    while True:
        user_input = input("You: ").strip()
        if user_input.lower() in ["exit", "quit"]:
            print("👋 Exiting LawBot. Goodbye!")
            break

        print("Thinking...\n")
        try:
            response = rag_chat(user_input)
            print(f"📜 LawBot: {response}\n")
        except Exception as e:
            print(f"Error: {str(e)}\n")


run_chat()


🧑‍⚖️ LawBot RAG Chat (type 'exit' to quit)

You: hey
Thinking...

📜 LawBot: Hello! How can I assist you today?

You: what are you??
Thinking...

📜 LawBot: I am an AI language model and a legal assistant.

You: an a magistrate take cognizance on a police report without sanction under Sec 197 CrPC?
Thinking...

📜 LawBot: No, a magistrate cannot take cognizance of a police report without sanction under Section 197 of the CrPC. Section 197 provides that no court or tribunal shall take cognizance of any offence except upon a police report filed by an officer in charge of a police station, or upon an order from a competent authority. Therefore, before a magistrate can take cognizance of a police report, it must first be filed by an officer in charge of a police station and approved by a competent authority.

You: Explain what happened in Maneka Gandhi v. Union of India
Thinking...

📜 LawBot: In Maneka Gandhi v. Union of India, the Supreme Court of India considered whether the government's de